# Topic Modeling

Christoph & Nadia

## LDA (Latent Dirichlet Allocation)

## Imports von hauptsächlich Gensim & SpaCy

In [1]:
from pprint import pprint
import gensim
from gensim import corpora, models, similarities
from gensim.models import LdaModel, LsiModel
from spacy_preprocessing.preprocess import Preprocess
from tabulate import tabulate

import numpy as np
import pickle
import pyLDAvis
import pyLDAvis.gensim

import os
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import widgets
from IPython.display import display, clear_output

from spacy_preprocessing.preprocess import Preprocess
import datetime

load_saved = False

## Zusätzliche Preprocessing Funktionen

In [2]:
def is_blacklisted(word):
    return word in [
        'polizei', 'polizist', 'beamter', 'nr.', 'berlin', 'uhr', 'polizeimeldung',
        'nicht', 'jahr', 'jährige', 'jährig', 'jähriger', 'polizeiliche', 'polizeilich', '2015', '2016',
        '2014', '2017', '2018', 'polizeibeamter', '-', 'u.a.', 'z.b.', 'der', 'die', 'das', 'dem', 'den', 'diese', 'dieser',
        'diesen', 'diesem', 'um', 'für', 'eine', 'ein', 'einer', 'einen', 'einem', 'anderer', 'andere', 'anderen', 'anders'
    ]

def is_empty(word):
    return word.strip() == ''

def can_parse_date(word):
    try:
        datetime.datetime.strptime(word, '%d.%m.%Y')
        return True
    except ValueError:
        return False

def can_parse_num_int(word):
    try:
        int(word)
        return True
    except ValueError:
        return False
    
def can_parse_num_float(word):
    try:
        float(word)
        return True
    except ValueError:
        return False
    
def preprocess_after(doc):
    return [
        word 
        for word in doc
        if not is_empty(word)
        and not is_blacklisted(word)
        and not can_parse_date(word)
        and not can_parse_num_int(word)
        and not can_parse_num_float(word)
    ]

## Daten aus pickle Datei laden und Preprocessing durchführen

In [3]:
items = pickle.load(open('./data/items.pkl', 'rb'))
texts = [report['text'] for report in items]
titles = [report['title'] for report in items]
ids = [report['_id'] for report in items]
urls = [report['url'] for report in items]
data = [report['text_pre_processed_v1'] for report in items]

In [4]:
clean_data = [preprocess_after(doc) for doc in data]

## Dictionary und Bag of Words aus jedem Dokument erstellen

In [5]:
if load_saved:
    id2word = corpora.Dictionary.load('./data/id2word')
    corpus = pickle.load(open('./data/corpus.pkl', 'rb'))
else:
    id2word = corpora.Dictionary(clean_data)
    corpus = [id2word.doc2bow(doc) for doc in clean_data]
    
    id2word.save('./data/id2word')
    pickle.dump(corpus, open('./data/corpus.pkl', 'wb'))

## LDA/LSI Modell

In [6]:
if load_saved:
    lda_model = LdaModel.load('./data/lda_model')
    lsi_model = LsiModel.load('./data/lsi_model')
else: 
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=15, passes=50, alpha='auto', eta='auto')
    lsi_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=15, power_iters=50)
    lda_model.save('./data/lda_model')
    lsi_model.save('./data/lsi_model')

In [7]:
pprint(lda_model.print_topics())

[(0,
  '0.029*"feuerwehr" + 0.028*"ermittlung" + 0.026*"alarmieren" + '
  '0.023*"landeskriminalamt" + 0.020*"löschen" + 0.019*"feuer" + '
  '0.018*"beschädigen" + 0.017*"nacht" + 0.015*"übernehmen" + 0.015*"brand"'),
 (1,
  '0.027*"flüchten" + 0.027*"geld" + 0.021*"angestellte" + 0.021*"ermittlung" '
  '+ 0.020*"täter" + 0.019*"unbekannte" + 0.018*"räuber" + 0.017*"richtung" + '
  '0.016*"betreten" + 0.016*"bedrohen"'),
 (2,
  '0.026*"pankow" + 0.019*"allee" + 0.017*"prenzlauer" + 0.016*"krankenhaus" + '
  '0.015*"berg" + 0.013*"rettungskraft" + 0.012*"ermittlung" + 0.012*"marzahn" '
  '+ 0.012*"alt" + 0.011*"behandlung"'),
 (3,
  '0.018*"km/h" + 0.010*"höhe" + 0.010*"verstoß" + 0.010*"böller" + '
  '0.009*"insgesamt" + 0.009*"bußgeld" + 0.009*"euro" + 0.008*"fahrverbot" + '
  '0.008*"punkt" + 0.008*"erlauben"'),
 (4,
  '0.028*"fahrer" + 0.028*"fahrzeug" + 0.024*"auto" + 0.022*"wagen" + '
  '0.021*"fahren" + 0.010*"richtung" + 0.009*"audi" + 0.009*"stehen" + '
  '0.008*"straße" + 0.00

## Topics visualisieren

In [8]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [9]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.147176 -0.218822       1        1  16.196669
7      0.058302  0.177666       2        1  13.675280
1      0.159969  0.078510       3        1   9.855567
14     0.097727  0.105591       4        1   9.854272
13     0.204348  0.012323       5        1   8.477991
0      0.060267 -0.012046       6        1   7.949502
4      0.032631 -0.140669       7        1   7.469129
6      0.151235  0.125770       8        1   5.722765
10    -0.083297  0.134745       9        1   4.797584
2      0.119182 -0.203512      10        1   4.303513
11    -0.092053  0.007111      11        1   2.796824
5     -0.179889  0.138356      12        1   2.642484
3     -0.216758 -0.096923      13        1   2.515439
9     -0.248621 -0.000903      14        1   2.450510
8     -0.210219 -0.107198      15        1   1.292476, topic_info=      Category         Freq                   Term        Total  loglift  \
term                                                                       
9      Default  3746.000000            krankenhaus  3746.000000  30.0000   
49     Default  5093.000000                 straße  5093.000000  29.0000   
159    Default  9018.000000                   mann  9018.000000  28.0000   
33     Default  1586.000000                   geld  1586.000000  27.0000   
209    Default  2484.000000              stationär  2484.000000  26.0000   
251    Default  1539.000000                 fahrer  1539.000000  25.0000   
70     Default  2180.000000              feuerwehr  2180.000000  24.0000   
631    Default  2082.000000               fahrzeug  2082.000000  23.0000   
207    Default  3328.000000                 schwer  3328.000000  22.0000   
347    Default  1866.000000             mutmaßlich  1866.000000  21.0000   
7      Default  3017.000000               flüchten  3017.000000  20.0000   
229    Default  2190.000000                   auto  2190.000000  19.0000   
751    Default  1148.000000            angestellte  1148.000000  18.0000   
81     Default  2057.000000      landeskriminalamt  2057.000000  17.0000   
182    Default  2924.000000               erleiden  2924.000000  16.0000   
17     Default  2935.000000             unbekannte  2935.000000  15.0000   
136    Default  3210.000000                   frau  3210.000000  14.0000   
248    Default  1330.000000               erfassen  1330.000000  13.0000   
57     Default  3249.000000             alarmieren  3249.000000  12.0000   
2      Default  3127.000000             behandlung  3127.000000  11.0000   
185    Default  1722.000000             festnehmen  1722.000000  10.0000   
304    Default  1737.000000         verkehrsunfall  1737.000000   9.0000   
475    Default  1234.000000                  wagen  1234.000000   8.0000   
923    Default   866.000000                löschen   866.000000   7.0000   
758    Default  1119.000000                 räuber  1119.000000   6.0000   
69     Default   842.000000                  feuer   842.000000   5.0000   
19     Default  2438.000000             verletzung  2438.000000   4.0000   
141    Default   901.000000                 pankow   901.000000   3.0000   
14     Default  4493.000000               richtung  4493.000000   2.0000   
222    Default  1688.000000                wohnung  1688.000000   1.0000   
...        ...          ...                    ...          ...      ...   
11513  Topic15    19.664524        flottwellstraße    19.946335   4.3344   
984    Topic15    19.309813         silvesterparty    19.592243   4.3341   
5501   Topic15    19.321991               kurdisch    19.604782   4.3341   
5371   Topic15    19.008945               bewähren    19.290033   4.3339   
5382   Topic15    19.008223  handlungsempfehlungen    19.290024   4.3339   
5383   Topic15    18.985977                   jörg    19.268133   4.3339   
4339   Topic15    44.419525            perleberger    48.189320   4.2672   

## MatrixSimilarity erstellen

In [10]:
if load_saved:
    index = similarities.MatrixSimilarity.load('./data/index')
else:
    index = similarities.MatrixSimilarity(lda_model[corpus])
    index.save('./data/index')

## Die Suche

In [11]:
use_lsi = False

html_widget = widgets.HTML(
    value = ''
)

text_widget = widgets.Text()


def search(sender):  
    preprocess = Preprocess(sender.value)
    search_term_preprocessed = preprocess_after(preprocess.preprocess(sentence_split=False, with_pos=False))
    search_term_bow = id2word.doc2bow(search_term_preprocessed)
    
    if use_lsi:
        search_term_vec = lsi_model[search_term_bow]
    else:
        search_term_vec = lda_model[search_term_bow]
    
    
    sims = index[search_term_vec]
    similarity_list = list(zip(range(len(sims)), sims, texts, titles, ids, urls))
    similarity_list.sort(key=lambda x: x[1], reverse=True)
    
    results = [f'<h3>{result[3]}</h3><em>Id: {result[4]} | Similarity Score: {round(result[1] * 100, 2)} % | Link: <a target="_blank" href="{result[5]}">{result[5]}</a></em><br><p style="width: 80%;">{result[2]}</p>' for result in similarity_list[:10]]
    output = '<hr><br>'.join(results)
    
    html_widget.value = output

text_widget.on_submit(search)    
    
display(text_widget)
display(html_widget)    


Text(value='')

HTML(value='')